In [1]:
import glob
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# import train csv
df = pd.read_csv('train.csv')

In [ ]:
all_columns = list(df.columns.values)
feature_columns = all_columns[3:]

In [6]:
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

In [7]:
data_selection = df.sample(
    10000
)  # reduce by sampling N random rows, plotting else takes forever

In [8]:
def plot_pca_and_create_anim(ft):
    # get matching features for feature name (ft)
    matching_features = [f for f in feature_columns if ft in f]
    # get matching numpy data
    X = data_selection[matching_features].values
    # let a PCA run
    X = PCA(n_components=3).fit_transform(X)

    # create a 3d projection and scatter plot data
    fig = plt.figure(1, figsize=(20, 14))
    ax = fig.add_subplot(projection="3d", elev=10, azim=0)
    scatter = ax.scatter(
        X[:, 0],
        X[:, 1],
        X[:, 2],
        c=data_selection.Class,
        cmap=plt.cm.nipy_spectral,
        edgecolor="k",
    )
    # show classes belonging to hues
    ax.legend(*scatter.legend_elements())
    ax.set_axis_off()

    # show graphics
    plt.show()

    # create picture from every angle
    # for animation
    for angle in range(0, 360, 2):
        # set angle
        ax.azim = angle
        # save figures in temp folder
        fig.savefig(f'./tmp/{angle}.png')

    # use opencv to read the files
    img_array = []
    for filename in sorted(glob.glob('./tmp/*.png'), key=os.path.getmtime):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)


    # and then write them to a video
    out = cv2.VideoWriter(f'./movs/{ft}.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
